### data set
https://www.kaggle.com/retailrocket/ecommerce-dataset

In [1]:
import numpy as np
import pandas as pd

### load data

In [2]:
events = pd.DataFrame.from_csv('./data/train_events_1.csv')
item_properties = pd.DataFrame.from_csv('./data/train_item_properties.csv')
category_tree = pd.DataFrame.from_csv('./data/category_tree.csv')

In [3]:
events.shape[0]

1402995

In [4]:
events.head()

,index,timestamp,visitorid,event,itemid,transactionid,delta,delta1,ssid,session_type
0,2024720,1438969904567,2,view,325215,-1,NaN,NaN,1.0,2.0
1,2024760,1438970013790,2,view,325215,-1,1.820383,1.820383,1.0,2.0
2,2024834,1438970212664,2,view,259884,-1,3.314567,3.314567,1.0,2.0
3,2024945,1438970468920,2,view,216305,-1,4.270933,4.270933,1.0,2.0
4,2025140,1438970905669,2,view,342816,-1,7.279150,7.279150,1.0,2.0


In [5]:
item_properties.shape

(18022776, 4)

In [6]:
item_properties.head()

,timestamp,itemid,property,value
0,1431226800000,317951,790,n32880.000
1,1431226800000,422842,480,1133979
2,1431226800000,310185,776,103591
3,1431226800000,110973,112,679677
4,1431226800000,179597,available,0


In [7]:
category_tree.shape

(1669, 1)

In [8]:
category_tree.head()

,parentid
categoryid,
1016,213.0
809,169.0
570,9.0
1691,885.0
536,1691.0


In [9]:
category_tree.parentid.unique().shape[0]

363

In [10]:
category_tree.index.value_counts().shape[0]

1669

In [11]:
#выберем из item_properties категории обьектов

In [12]:
categories = item_properties.loc[item_properties.property =='categoryid'].sort_values([ 'itemid','timestamp'], ascending = [True,False])

In [13]:
categories.value.unique().shape

(1209,)

In [14]:
categories.head()

,timestamp,itemid,property,value
881951,1431226800000,0,categoryid,209
154130,1431226800000,1,categoryid,1114
1086786,1431226800000,2,categoryid,1305
7976913,1433041200000,3,categoryid,1171
3607914,1431831600000,4,categoryid,1038


In [15]:
events.shape[0] - events.loc[events.itemid.isin(categories.itemid.unique())].shape[0]

84980

In [16]:
#функция определиния катеогории объекта
def get_cat(timestamp,itemid):
    result = np.nan
    i_ = categories.loc[(categories.itemid == itemid)&(categories.timestamp <= timestamp)]
    if i_.shape[0] != 0:
        result = int(i_.iloc[0].value)
    return result

In [17]:
e__ = np.array(events[['timestamp','itemid']])

In [18]:
e__.shape

(1402995, 2)

In [19]:
#Внимание!!! Wall time: 51min 12s

In [20]:
%%time
cat = []
for i in e__:
    cat.append(get_cat(i[0],i[1]))

Wall time: 1h 13min


In [21]:
events['item_cat'] = cat

In [22]:
def get_cat_list(categoryid):
    result = [categoryid]
    cat = categoryid
    while True:
        #print(cat)
        if np.isnan(cat):
            break
        cat = category_tree.loc[category_tree.index == int(cat)].iloc[0]['parentid']
        result.append(cat)
    return result

def item_distance(cat1, cat2):
    if np.isnan(cat1) and np.isnan(cat2) :
        return 0
    
    if np.isnan(cat1) or np.isnan(cat2) :
        if np.isnan(cat1):
            return len(get_cat_list(cat2))
        if np.isnan(cat2):
            return len(get_cat_list(cat1))
        
    if cat1 == cat2 :
        return 0
        
    cat_list1 = get_cat_list(cat1)
    cat_list2 = get_cat_list(cat2)
    cat_list1.reverse()
    cat_list2.reverse()

    i = 0
    result = []
    while True:
        if (i >= len(cat_list1) or i >= len(cat_list2)):
            break
        result.append(cat_list1[i]-cat_list2[i])
        i = i +1
    r = 0

    for r_ in result:
        if r_ != 0:
            break
        r = r + 1
    return (len(cat_list1)- r + len(cat_list2)-r)
    

In [23]:
%%time

d_ = np.zeros(events.shape[0])
session = '0'
dist = events.iloc[0]['item_cat']
for index, row in events.iterrows():
    if session != row['ssid']:
        dist = row['item_cat']
    d_[index] = item_distance(row['item_cat'],dist)
    dist = row['item_cat'] 
    session = row['ssid']

Wall time: 10min 31s


In [24]:
events['item_cat_dist'] = d_

In [25]:
events.head(10)

,index,timestamp,visitorid,event,itemid,transactionid,delta,delta1,ssid,session_type,item_cat,item_cat_dist
0,2024720,1438969904567,2,view,325215,-1,NaN,NaN,1.0,2.0,299.0,0.0
1,2024760,1438970013790,2,view,325215,-1,1.820383,1.820383,1.0,2.0,299.0,0.0
2,2024834,1438970212664,2,view,259884,-1,3.314567,3.314567,1.0,2.0,299.0,0.0
3,2024945,1438970468920,2,view,216305,-1,4.270933,4.270933,1.0,2.0,299.0,0.0
4,2025140,1438970905669,2,view,342816,-1,7.279150,7.279150,1.0,2.0,444.0,10.0
5,2025338,1438971444375,2,view,342816,-1,8.978433,8.978433,1.0,2.0,444.0,0.0
6,2025347,1438971463170,2,view,216305,-1,0.313250,0.313250,1.0,2.0,299.0,10.0
7,2025417,1438971657845,2,view,325215,-1,3.244583,3.244583,1.0,2.0,299.0,0.0
8,216860,1431581976753,7,view,139394,-1,NaN,NaN,2.0,1.0,NaN,0.0
9,216903,1431582162817,7,view,164941,-1,3.101067,3.101067,2.0,1.0,1114.0,4.0


In [26]:
#save data
events.to_csv('./data/train_events_2.csv')

In [27]:
### load data
events = pd.DataFrame.from_csv('./data/train_events_2.csv')

In [28]:
events.head()

,index,timestamp,visitorid,event,itemid,transactionid,delta,delta1,ssid,session_type,item_cat,item_cat_dist
0,2024720,1438969904567,2,view,325215,-1,NaN,NaN,1.0,2.0,299.0,0.0
1,2024760,1438970013790,2,view,325215,-1,1.820383,1.820383,1.0,2.0,299.0,0.0
2,2024834,1438970212664,2,view,259884,-1,3.314567,3.314567,1.0,2.0,299.0,0.0
3,2024945,1438970468920,2,view,216305,-1,4.270933,4.270933,1.0,2.0,299.0,0.0
4,2025140,1438970905669,2,view,342816,-1,7.279150,7.279150,1.0,2.0,444.0,10.0


In [29]:
##уберем лишние данные и "причешем" дата сет

In [30]:
events['event_type'] = events.event.copy()

In [31]:
events.event_type = events.event_type.replace({'view': 0, 'addtocart': 1, 'transaction': 2})

In [32]:
events['event_type'] = events['event_type'].astype(int)

In [33]:
events['session_type'] = events['session_type'].fillna(-1).astype(int)

In [34]:
events['ssid'] = events['ssid'].astype(int)

In [35]:
events.head()

,index,timestamp,visitorid,event,itemid,transactionid,delta,delta1,ssid,session_type,item_cat,item_cat_dist,event_type
0,2024720,1438969904567,2,view,325215,-1,NaN,NaN,1,2,299.0,0.0,0
1,2024760,1438970013790,2,view,325215,-1,1.820383,1.820383,1,2,299.0,0.0,0
2,2024834,1438970212664,2,view,259884,-1,3.314567,3.314567,1,2,299.0,0.0,0
3,2024945,1438970468920,2,view,216305,-1,4.270933,4.270933,1,2,299.0,0.0,0
4,2025140,1438970905669,2,view,342816,-1,7.279150,7.279150,1,2,444.0,10.0,0


In [36]:
## теперь отбросим все события в сессих после покупки

In [37]:
events_trans = events.sort_values(['ssid', 'timestamp'], ascending=[True, True])

In [38]:
ssids = events_trans.ssid.unique()

In [39]:
%%time
drop_list = []
transaction_list = []
for i in ssids:
    event = events_trans.loc[events_trans.ssid == i]
    for index, row in event.iterrows():
        if row['event_type'] == 2:
            drop_list.extend(list(event.loc[event.index > index].index))
            transaction_list.append(i)
            break

Wall time: 28min 2s


In [40]:
len(transaction_list) 

11200

In [41]:
len(drop_list)

47172

In [42]:
e = events.loc[events.index.isin(drop_list)]

In [43]:
e.shape

(47172, 13)

In [44]:
events.shape[0] - e.shape[0]

1355823

In [45]:
events = events.drop(e.index,axis=0)

In [46]:
events.shape[0]

1355823

In [47]:
events.reset_index(inplace=True)

In [48]:
events = events.drop('level_0',axis=1)

In [49]:
events_trans = events.loc[events.ssid.isin(transaction_list)]

In [50]:
events_trans.shape

(63475, 13)

In [51]:
events['session_transation_type'] = 0

In [52]:
events['session_transation_type'].loc[events.ssid.isin(transaction_list)] = 1

C:\Users\admin\AppData\Local\conda\conda\envs\DS-1\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [53]:
events['session_transation_type'].value_counts()

0    1292348
1      63475
Name: session_transation_type, dtype: int64

### Итоговый дата сет

In [54]:
#количество событий в сете для классификации
events.shape[0]

1355823

In [58]:
events.head()

,index,timestamp,visitorid,event,itemid,transactionid,delta,delta1,ssid,session_type,item_cat,item_cat_dist,event_type,session_transation_type
0,2024720,1438969904567,2,view,325215,-1,NaN,NaN,1,2,299.0,0.0,0,0
1,2024760,1438970013790,2,view,325215,-1,1.820383,1.820383,1,2,299.0,0.0,0,0
2,2024834,1438970212664,2,view,259884,-1,3.314567,3.314567,1,2,299.0,0.0,0,0
3,2024945,1438970468920,2,view,216305,-1,4.270933,4.270933,1,2,299.0,0.0,0,0
4,2025140,1438970905669,2,view,342816,-1,7.279150,7.279150,1,2,444.0,10.0,0,0


In [56]:
events = events.sort_values(['ssid', 'timestamp'], ascending=[True, True])

In [57]:
#save data
events.to_csv('./data/train_events_3.csv')